# Audio Transcriptions 

The goal of this notebook is to transcribe the audio files of the podcass using a model.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')


In [6]:

client = OpenAI()

audio_file = open("../audios/2025_06_18_19.mp3", "rb")
stream = client.audio.transcriptions.create(
  file=audio_file,
  model="gpt-4o-mini-transcribe",
  stream=True,
  language="es"
)

transcription_text = ""

for event in stream:
    print(event)
    if hasattr(event, 'text') and event.text:
        transcription_text += event.text

# Guardar la transcripción en un archivo txt
with open("../transcriptions/transcription_2025_06_18_19.txt", "w", encoding="utf-8") as f:
    f.write(transcription_text)

print(f"✅ Transcripción guardada en: ../transcriptions/transcription_2025_06_18_19.txt")

TranscriptionTextDeltaEvent(delta='Ser', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' Podcast', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta='.', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' En', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' la', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' ventana', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=',', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' acontece', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' que', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' no', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' es', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' poco'